In [99]:
import torch

from datasets import LabelledPointCloudDataset
from models import LabelledPointNet, Decoder, LabelledPointNetAE

import csv
import json
import os
from pathlib import Path

In [100]:
device = "cuda" if torch.cuda.is_available() else "cpu"
n = 8192
c = 19

dataset = LabelledPointCloudDataset("./data/test/", c, 100, max_points=n)
loader = torch.utils.data.DataLoader(dataset, 1, shuffle=False)

model = LabelledPointNetAE(n, c)
model.load_state_dict(torch.load(
    "./runs/laptop/model-state-dict.pt", weights_only=True, map_location=torch.device(device)
))
model.to(device)
model.eval()

LabelledPointNetAE(
  (encoder): LabelledPointNet(
    (tnet1): InputTNet(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU()
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (tnet2): FeatureTNet(
      (conv1): Conv

In [101]:
idx = 0
for batch in loader:
    x = batch.to(device)
    out = model(x)
    print(out)
    

tensor([[[ 1.5244e-01,  1.5364e-01, -3.3755e-02,  ...,  1.7815e-01,
           1.1703e-01,  8.6749e-02],
         [ 2.0263e-01,  1.5749e-01,  1.7854e-01,  ...,  8.6802e-02,
           1.9532e-01,  2.1607e-01],
         [-4.4239e-02, -1.3939e-01, -1.2608e-01,  ..., -3.7756e-03,
          -1.2818e-01, -1.0141e-01],
         ...,
         [-7.4046e+00, -6.7301e+00, -7.2872e+00,  ..., -7.4599e+00,
          -7.3725e+00, -8.3644e+00],
         [-1.0519e+01, -1.0474e+01, -1.0265e+01,  ..., -1.0726e+01,
          -1.0572e+01, -1.0548e+01],
         [-8.7709e+00, -8.2171e+00, -8.8317e+00,  ..., -9.4423e+00,
          -7.9894e+00, -8.6191e+00]]], device='cuda:0', grad_fn=<CatBackward0>)


In [102]:
idx = 0
points = out[idx][:3, :]
points.T * 100
labels = out[idx][3:, :]
with open(f"{5}.csv", 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(torch.cat(((points.T * 100), torch.max(labels, dim=0).indices.unsqueeze(1)), dim=1).cpu().tolist())